In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [ ]:
import torch

In [ ]:
tweets = pd.read_parquet("stock_tweet_no_merge.parquet")

In [ ]:
tweets

In [ ]:
tweets.shape

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
bert_model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
device = torch.device("cuda")
bert_model = bert_model.to(device)

In [ ]:
torch.cuda.is_available()
print(next(bert_model.parameters()).device)

In [ ]:
def cal_sentiment_score(text):
    inputs = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = bert_model(inputs)
    
    sentiment = int(torch.argmax(outputs.logits)) + 1
    return sentiment

In [ ]:
tweets['sentiment'] = tweets['text'].apply(cal_sentiment_score)

In [ ]:
tweets

In [ ]:
tweets.to_parquet('stock_tweets_withsentiment_nomerge',index=False)